# Import Library

In [1]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import numpy as np
# np.float_ = np.float64

from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from statsmodels.tsa.holtwinters import ExponentialSmoothing

import pmdarima as pmd

# Import Data

## [Electric Vehicle Population Data](https://catalog.data.gov/dataset/electric-vehicle-population-data)
This dataset shows the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through Washington State Department of Licensing (DOL).

In [2]:
df = pd.read_csv("./Electric_Vehicle_Population_Data.csv", header=0)

## Quickview of the data

In [3]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1C4RJXN66R,Snohomish,Everett,WA,98204.0,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,0.0,21.0,261311557,POINT (-122.2507211 47.8976713),PUGET SOUND ENERGY INC,5.306104e+10
1,KNDJX3AEXG,King,Renton,WA,98058.0,2016,KIA,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,11.0,210641315,POINT (-122.1476337 47.4438471),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
2,5YJ3E1EA3L,King,Seattle,WA,98125.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266.0,0.0,46.0,124517347,POINT (-122.304356 47.715668),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,1G1RC6S5XH,Kitsap,Port Orchard,WA,98367.0,2017,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,53.0,0.0,26.0,7832933,POINT (-122.6530052 47.4739066),PUGET SOUND ENERGY INC,5.303509e+10
4,5UXTA6C09P,Snohomish,Monroe,WA,98272.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30.0,0.0,39.0,235249262,POINT (-121.968385 47.854897),PUGET SOUND ENERGY INC,5.306105e+10


## Data Structure

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194232 entries, 0 to 194231
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         194232 non-null  object 
 1   County                                             194223 non-null  object 
 2   City                                               194223 non-null  object 
 3   State                                              194232 non-null  object 
 4   Postal Code                                        194223 non-null  float64
 5   Model Year                                         194232 non-null  int64  
 6   Make                                               194232 non-null  object 
 7   Model                                              194232 non-null  object 
 8   Electric Vehicle Type                              194232 non-null  object

# Data Exploration

## Unique values

In [5]:
print("Unique Values in 'VIN (1-10)':", df['VIN (1-10)'].unique())

Unique Values in 'VIN (1-10)': ['1C4RJXN66R' 'KNDJX3AEXG' '5YJ3E1EA3L' ... 'YV4H60DK1R' '1GYKPMRL9R'
 'WP0CD2Y10N']


In [6]:
print("Unique Values in 'County':", df['County'].unique())

Unique Values in 'County': ['Snohomish' 'King' 'Kitsap' 'Yakima' 'Thurston' 'Stevens' 'Chelan'
 'Island' 'Spokane' 'Douglas' 'Whitman' nan 'Columbia' 'Walla Walla'
 'Kittitas' 'Skagit' 'Grant' 'Clark' 'Cowlitz' 'Jefferson' 'Clallam'
 'Klickitat' 'Benton' 'Whatcom' 'Pierce' 'Mason' 'Lewis' 'Franklin'
 'Okanogan' 'Asotin' 'Grays Harbor' 'Marin' 'Adams' 'Wahkiakum' 'San Juan'
 'Skamania' 'Montgomery' 'Pacific' 'Ferry' 'Pend Oreille' 'Sacramento'
 'Lincoln' 'York' 'District of Columbia' 'Orange' 'Stafford' 'San Diego'
 'Churchill' 'Richland' 'DeKalb' 'Oldham' 'Washoe' 'Fairfax' 'Kern'
 'Los Angeles' 'Sarasota' 'Maricopa' 'Maui' 'Kings' 'Lee' 'Riverside'
 'Pulaski' 'Cumberland' 'San Mateo' 'DuPage' 'El Paso' 'Multnomah'
 'Garfield' 'Honolulu' 'Alameda' 'New London' 'Isle of Wight'
 'Contra Costa' 'Ada' 'Miami-Dade' 'Anne Arundel' 'Collin' 'New York'
 'Portsmouth' 'Ventura' 'Loudoun' 'Santa Barbara' 'Carroll' 'Solano'
 'Sonoma' 'Harnett' 'Monterey' 'James City' 'Beaufort' 'Leavenworth'
 'Mob

In [7]:
print("Unique Values in 'City':", df['City'].unique())

Unique Values in 'City': ['Everett' 'Renton' 'Seattle' 'Port Orchard' 'Monroe' 'Moxee' 'Olympia'
 'Mill Creek' 'Bremerton' 'Arlington' 'Rochester' 'Auburn' 'Bothell'
 'Bainbridge Island' 'Poulsbo' 'Tumwater' 'Snohomish' 'Colville' 'Yakima'
 'Yelm' 'Seatac' 'Wenatchee' 'Silverdale' 'Selah' 'Kingston' 'Kirkland'
 'Hansville' 'Shoreline' 'Lacey' 'Redmond' 'Lake Stevens' 'Seabeck'
 'Bellevue' 'Union Gap' 'Manson' 'Mukilteo' 'Marysville' 'Naches'
 'Oak Harbor' 'Issaquah' 'Freeland' 'Burien' 'Toppenish' 'Langley'
 'Spokane' 'Edmonds' 'East Wenatchee' 'Sultan' 'Pullman' nan 'Lynnwood'
 'Dayton' 'Walla Walla' 'Ronald' 'Anacortes' 'Moses Lake' 'Wapato'
 'Clinton' 'North Bend' 'Vancouver' 'Lake Forest Park' 'Silverlake'
 'Kalama' 'Sammamish' 'Woodinville' 'Newcastle' 'Duvall' 'Yacolt' 'Kent'
 'Federal Way' 'Port Townsend' 'Kelso' 'Tukwila' 'Longview'
 'Normandy Park' 'Kenmore' 'Clyde Hill' 'Beaux Arts' 'Mercer Island'
 'Camano Island' 'Covington' 'Battle Ground' 'Olalla' 'Camas' 'Sequim'
 'Washo

In [8]:
print("Unique Values in 'State':", df['State'].unique())

Unique Values in 'State': ['WA' 'BC' 'CA' 'MD' 'VA' 'DC' 'NV' 'SC' 'GA' 'KY' 'NE' 'FL' 'AZ' 'HI'
 'AL' 'MO' 'PA' 'IL' 'TX' 'OR' 'KS' 'CT' 'ID' 'CO' 'NY' 'NC' 'AE' 'UT'
 'NM' 'MI' 'LA' 'OH' 'WI' 'IN' 'DE' 'MT' 'AR' 'NJ' 'RI' 'MA' 'OK' 'MN'
 'AK' 'IA' 'WY' 'NH']


In [9]:
print("Unique Values in 'Make':", df['Make'].unique())

Unique Values in 'Make': ['JEEP' 'KIA' 'TESLA' 'CHEVROLET' 'BMW' 'FORD' 'NISSAN' 'PORSCHE' 'VOLVO'
 'MINI' 'TOYOTA' 'AUDI' 'FIAT' 'JAGUAR' 'HYUNDAI' 'POLESTAR'
 'MERCEDES-BENZ' 'LUCID' 'MAZDA' 'VOLKSWAGEN' 'FISKER' 'RIVIAN'
 'MITSUBISHI' 'HONDA' 'CHRYSLER' 'SUBARU' 'LINCOLN' 'SMART' 'LEXUS'
 'CADILLAC' 'GENESIS' 'LAND ROVER' 'ALFA ROMEO' 'DODGE' 'TH!NK' 'GMC'
 'BENTLEY' 'ROLLS-ROYCE' 'ACURA' 'WHEEGO ELECTRIC CARS' 'AZURE DYNAMICS'
 'RAM']


In [10]:
print("Unique Values in 'Model':", df['Model'].unique())

Unique Values in 'Model': ['WRANGLER' 'SOUL' 'MODEL 3' 'VOLT' 'X5' 'ESCAPE' 'MODEL S' 'LEAF'
 'CAYENNE' 'X3' 'MODEL Y' 'MODEL X' 'XC90' 'BOLT EV' 'I3' 'F-150'
 'COUNTRYMAN' 'NIRO' 'PRIUS PRIME' 'Q5' 'EV6' 'BOLT EUV' 'Q5 E'
 'RAV4 PRIME' 'HARDTOP' '500' '330E' 'E-TRON' '530E' 'FUSION'
 'GRAND CHEROKEE' 'MUSTANG MACH-E' 'I-PACE' 'KONA' 'S60' 'PS2' 'GLC-CLASS'
 'AIR' 'IX' 'CX-90' 'XC60' 'ID.4' 'OCEAN' 'EV9' 'SORENTO' 'EQS-CLASS SUV'
 'R1S' 'SPORTAGE' 'OUTLANDER' 'CLARITY' 'FOCUS' 'E-GOLF' 'TUCSON' 'R1T'
 'TAYCAN' 'PANAMERA' 'Q4' 'PACIFICA' 'EQE-CLASS SEDAN' 'I4' 'SOLTERRA'
 'IONIQ 5' 'EQE-CLASS SUV' 'KONA ELECTRIC' 'BZ4X' 'AVIATOR' 'A3' 'C-MAX'
 'XC40' 'EQ FORTWO' 'EQB-CLASS' 'NX' 'IONIQ' 'PRIUS PLUG-IN' 'SANTA FE'
 'ELR' 'CYBERTRUCK' 'ARIYA' 'FORTWO ELECTRIC DRIVE' 'XM' 'RZ' 'B-CLASS'
 'C40' 'SONATA' 'I-MIEV' 'I8' 'Q8' 'SPARK' 'IONIQ 6' 'TRANSIT' 'BLAZER EV'
 'RANGER' 'GLE-CLASS' 'E-TRON SPORTBACK' 'E-TRON GT' 'I7' 'IONIQ 5 N'
 'ROADSTER' 'GV70' 'CORSAIR' 'I5' 'RANGE ROVER SPORT' 'LYRIQ'

In [11]:
print("Unique Values in 'Electric Vehicle Type':", df['Electric Vehicle Type'].unique())

Unique Values in 'Electric Vehicle Type': ['Plug-in Hybrid Electric Vehicle (PHEV)' 'Battery Electric Vehicle (BEV)']


## Summary of data

In [12]:
df.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract
count,194223.000000,194232.000000,194230.000000,194230.000000,193800.000000,1.942320e+05,1.942230e+05
mean,98175.800678,2020.781807,54.835458,978.730732,29.009954,2.248923e+08,5.297532e+10
std,2435.345863,2.999041,89.614355,7988.719011,14.901335,7.357830e+07,1.607770e+09
min,1731.000000,1997.000000,0.000000,0.000000,1.000000,4.385000e+03,1.001020e+09
25%,98052.000000,2019.000000,0.000000,0.000000,17.000000,1.872251e+08,5.303301e+10
50%,98125.000000,2022.000000,0.000000,0.000000,33.000000,2.339402e+08,5.303303e+10
75%,98372.000000,2023.000000,68.000000,0.000000,42.000000,2.601159e+08,5.305307e+10
max,99577.000000,2025.000000,337.000000,845000.000000,49.000000,4.792548e+08,5.602100e+10


## Grouped Summary by vehicle type

In [13]:
grouped_data = df[['Electric Vehicle Type', "Electric Range", "Base MSRP", "Legislative District", "2020 Census Tract"]]\
    .groupby('Electric Vehicle Type')  
print("\nGrouped Mean:")
grouped_data = grouped_data.describe().unstack(1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(grouped_data)


Grouped Mean:
                             Electric Vehicle Type                 
Electric Range        count  Battery Electric Vehicle (BEV)            1.521790e+05
                             Plug-in Hybrid Electric Vehicle (PHEV)    4.205100e+04
                      mean   Battery Electric Vehicle (BEV)            6.145813e+01
                             Plug-in Hybrid Electric Vehicle (PHEV)    3.086856e+01
                      std    Battery Electric Vehicle (BEV)            9.991519e+01
                             Plug-in Hybrid Electric Vehicle (PHEV)    1.524458e+01
                      min    Battery Electric Vehicle (BEV)            0.000000e+00
                             Plug-in Hybrid Electric Vehicle (PHEV)    6.000000e+00
                      25%    Battery Electric Vehicle (BEV)            0.000000e+00
                             Plug-in Hybrid Electric Vehicle (PHEV)    2.100000e+01
                      50%    Battery Electric Vehicle (BEV)            0.0000

## Visualization

### Range by vehicle type

In [14]:
fig = px.histogram(df, x = 'Electric Range', color = 'Electric Vehicle Type', histnorm='probability density')
fig.update_layout(height=500, width=1000, title='Range of vehicles')
fig.show()

### Electric vehicle usage over the years

In [15]:
grouped_df = df[['Model Year', 'Electric Vehicle Type']].assign(count = 1).groupby(['Model Year', 'Electric Vehicle Type']).\
    count().reset_index()
fig = px.bar(grouped_df, x = 'Model Year', y= 'count', color = "Electric Vehicle Type", barmode='group')
fig.update_layout(height=500, width=1000, title='EV Types over the years')
fig.show()

### Share of different EV Makers

In [16]:
grouped_df = df[['Make']].assign(count=1).groupby(['Make']).sum().reset_index()
grouped_df['percentage'] = (grouped_df['count'] / grouped_df['count'].sum()) * 100
others_makes = grouped_df[grouped_df['percentage'] < 2]['Make'].tolist()
grouped_df.loc[grouped_df['Make'].isin(others_makes), 'Make'] = 'Others'
grouped_df = grouped_df.groupby('Make').sum().reset_index()

fig = px.pie(grouped_df, names='Make', values='count')
fig.update_layout(height=500, width=1000, title='Market Share of EV Makers')
fig.show()

### Share of EV types from different Makers over the years

In [17]:
grouped_df = df[['Model Year', 'Electric Vehicle Type', 'Make']].assign(count = 1).\
    groupby(['Model Year', 'Make', 'Electric Vehicle Type']).count().reset_index()

grouped_df_1 = grouped_df[['Model Year', 'Electric Vehicle Type', 'count']].groupby(['Model Year', 'Electric Vehicle Type']).\
    sum().reset_index().rename(columns={'count': 'total'})

grouped_df_new = pd.merge(grouped_df, grouped_df_1, on=['Model Year', 'Electric Vehicle Type'], how='left')
grouped_df_new['percentage'] = (grouped_df_new['count'] / grouped_df_new['total']) * 100
grouped_df_new.loc[grouped_df_new['percentage'] < 40, 'Make'] = 'Others'

grouped_df_new = grouped_df_new[['Model Year', 'Make', 'Electric Vehicle Type', 'count']].\
    groupby(['Model Year', 'Make', 'Electric Vehicle Type']).sum().reset_index()
grouped_df_new = pd.merge(grouped_df_new, grouped_df_1, on=['Model Year', 'Electric Vehicle Type'], how='left')
grouped_df_new['percentage'] = (grouped_df_new['count'] / grouped_df_new['total']) * 100

fig = px.scatter_3d(grouped_df_new, x='Model Year', y='Make', z='count', size='percentage', color='Electric Vehicle Type')
fig.update_layout(scene_zaxis_type="log", height=500, width=1000, title='Bubble Plot of EV Share')
fig.show()

# Time series modelling of Vehicle presence on road by EV type

## Model - Prophet

In [18]:
model_df = df[['Model Year', 'Electric Vehicle Type']].assign(count = 1).groupby(['Model Year', 'Electric Vehicle Type']).\
    count().reset_index().rename(columns = {'Model Year': 'ds', 'count': 'y'})
model_df = model_df[model_df['ds'] < 2024]
model_df['ds'] = pd.to_datetime(model_df['ds'], format='%Y')
model_df.tail()

,ds,Electric Vehicle Type,y
29,2021-01-01,Plug-in Hybrid Electric Vehicle (PHEV),3922
30,2022-01-01,Battery Electric Vehicle (BEV),23730
31,2022-01-01,Plug-in Hybrid Electric Vehicle (PHEV),4426
32,2023-01-01,Battery Electric Vehicle (BEV),52165
33,2023-01-01,Plug-in Hybrid Electric Vehicle (PHEV),7500


### Plug-in Hybrid Electric Vehicle (PHEV)

#### Filter data

In [19]:
model_df_phev = model_df[model_df['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)'][['ds', 'y']].\
    sort_values(by = 'ds')
model_df_phev.tail()

,ds,y
25,2019-01-01,2031
27,2020-01-01,1834
29,2021-01-01,3922
31,2022-01-01,4426
33,2023-01-01,7500


#### Fitting Model

In [20]:
m = Prophet()
m.fit(model_df_phev)

max_year = model_df_phev['ds'].max()
min_year = model_df_phev['ds'].min()
date_range_freq = pd.date_range(start = min_year, end = max_year + pd.DateOffset(years=5), freq='YS')
future = pd.DataFrame({'ds': date_range_freq})
forecast = m.predict(future)
plot_plotly(m, forecast).update_layout(height=500, width=1000, \
                                       title = "Plug-in Hybrid Electric Vehicle (PHEV) On Road 5 Years Projection")

16:54:22 - cmdstanpy - INFO - Chain [1] start processing
16:54:22 - cmdstanpy - INFO - Chain [1] done processing


#### Forecast components

In [21]:
plot_components_plotly(m, forecast).update_layout(height=500, width=1000, title = "Components")

### Battery Electric Vehicle (BEV)

#### Filter Data

In [22]:
model_df_bev = model_df[model_df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)'][['ds', 'y']].\
    sort_values(by = 'ds')
model_df_bev.tail()

,ds,y
24,2019-01-01,8875
26,2020-01-01,10255
28,2021-01-01,15394
30,2022-01-01,23730
32,2023-01-01,52165


#### Fitting Model

In [23]:
m = Prophet()
m.fit(model_df_bev)

max_year = model_df_bev['ds'].max()
min_year = model_df_bev['ds'].min()
date_range_freq = pd.date_range(start = min_year, end = max_year + pd.DateOffset(years=5), freq='YS')
future = pd.DataFrame({'ds': date_range_freq})
forecast = m.predict(future)
plot_plotly(m, forecast).update_layout(height=500, width=1000, \
                                       title = "Battery Electric Vehicle (BEV) On Road 5 Years Projection")

16:54:22 - cmdstanpy - INFO - Chain [1] start processing
16:54:22 - cmdstanpy - INFO - Chain [1] done processing


#### Forecast Components

In [24]:
plot_components_plotly(m, forecast).update_layout(height=500, width=1000, title = "Components")

## Model - SARIMA

In [25]:
model_df = df[['Model Year', 'Electric Vehicle Type']].assign(count = 1).groupby(['Model Year', 'Electric Vehicle Type']).\
    count().reset_index().rename(columns = {'Model Year': 'ds', 'count': 'y'})
model_df = model_df[model_df['ds'] < 2024]
model_df['ds'] = pd.to_datetime(model_df['ds'], format='%Y')
model_df.tail()

,ds,Electric Vehicle Type,y
29,2021-01-01,Plug-in Hybrid Electric Vehicle (PHEV),3922
30,2022-01-01,Battery Electric Vehicle (BEV),23730
31,2022-01-01,Plug-in Hybrid Electric Vehicle (PHEV),4426
32,2023-01-01,Battery Electric Vehicle (BEV),52165
33,2023-01-01,Plug-in Hybrid Electric Vehicle (PHEV),7500


### Plug-in Hybrid Electric Vehicle (PHEV)

#### Filter data

In [26]:
model_df_phev = model_df[model_df['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)'][['ds', 'y']].\
    sort_values(by = 'ds')
model_df_phev.tail()

,ds,y
25,2019-01-01,2031
27,2020-01-01,1834
29,2021-01-01,3922
31,2022-01-01,4426
33,2023-01-01,7500


#### ADF test

In [27]:
p_value = adfuller(model_df_phev['y'], autolag = 'AIC')[1]
p_value

0.4981364528007875

As pvalue = 0.497 which is greater than 0.05, so we accept null hypothesis of ADF test true. Hence, the data is not stationary.

#### Detecting finest model

In [28]:
model=pmd.auto_arima(model_df_phev['y'],start_p=1,start_q=1, start_P = 1, start_Q = 1, \
                     test = 'adf', seasonal = True, trace = True, seasonal_test = "oscb", information_criterion = 'aicc')

Performing stepwise search to minimize aicc
 ARIMA(1,2,1)(0,0,0)[0]             : AICC=inf, Time=0.06 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AICC=215.228, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AICC=217.649, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AICC=inf, Time=0.05 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AICC=217.728, Time=0.00 sec

Best model:  ARIMA(0,2,0)(0,0,0)[0]          
Total fit time: 0.146 seconds


#### Fitting model

In [29]:
sarima = SARIMAX(model_df_phev['y'], order = (0,2,0), seasonal_order = (0,0,0,0))
sarima_model = sarima.fit()

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



#### Forecast

In [30]:
# predicted value
predicted = sarima_model.predict()
# predicted = predicted[1:]
# Forecast
forecast_periods = 5  
forecast = sarima_model.get_forecast(steps = forecast_periods) 
forecast_mean = forecast.predicted_mean 
# forecast_ci = forecast.conf_int() 

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



#### Visualize forecast

In [31]:
# Create a figure
fig = go.Figure()

# Add the observed data line
fig.add_trace(go.Scatter(
    x=list(range(len(model_df_phev['y'].tolist()))),
    y=model_df_phev['y'].tolist(),
    mode='lines',
    name='Observed'
))

# Add the forecast data line, starting right after the observed data
fig.add_trace(go.Scatter(
    x=list(range(len(predicted.tolist() + forecast_mean.tolist()))),
    y=predicted.tolist() + forecast_mean.tolist(),
    mode='lines',
    line=dict(color='red'),
    name='Forecast'
))

fig.update_layout(height=500, width=1000, title = "Plug-in Hybrid Electric Vehicle (PHEV) On Road 5 Years Projection")

fig.show()


### Battery Electric Vehicle (BEV)

#### Filter Data

In [32]:
model_df_bev = model_df[model_df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)'][['ds', 'y']].\
    sort_values(by = 'ds')
model_df_bev.tail()

,ds,y
24,2019-01-01,8875
26,2020-01-01,10255
28,2021-01-01,15394
30,2022-01-01,23730
32,2023-01-01,52165


#### ADF test

In [33]:
p_value = adfuller(model_df_bev['y'], autolag = 'AIC')[1]
p_value

1.0

As p-value = 1 which is greater than 0.05, so we accept null hypothesis i.e. the data is non-stationary.

#### Detecting finest model

In [34]:
model=pmd.auto_arima(model_df_bev['y'],start_p=1,start_q=1, start_P = 1, start_Q = 1, \
                     test = 'adf', seasonal = True, trace = True, seasonal_test = "oscb", information_criterion = 'aicc')

Performing stepwise search to minimize aicc
 ARIMA(1,2,1)(0,0,0)[0]             : AICC=366.720, Time=0.07 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AICC=362.745, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AICC=365.161, Time=0.01 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AICC=365.184, Time=0.04 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AICC=363.622, Time=0.01 sec

Best model:  ARIMA(0,2,0)(0,0,0)[0]          
Total fit time: 0.150 seconds


#### Fitting model

In [35]:
sarima = SARIMAX(model_df_bev['y'], order = (0,2,0), seasonal_order = (0,0,0,0))
sarima_model = sarima.fit()

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



#### Forecast

In [36]:
# predicted value
predicted = sarima_model.predict()
# predicted = predicted[1:]
# Forecast
forecast_periods = 5
forecast = sarima_model.get_forecast(steps = forecast_periods) 
forecast_mean = forecast.predicted_mean 
# forecast_ci = forecast.conf_int() 

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



#### Visualize forecast

In [37]:
# Create a figure
fig = go.Figure()

# Add the observed data line
fig.add_trace(go.Scatter(
    x=list(range(len(model_df_bev['y'].tolist()))),
    y=model_df_bev['y'].tolist(),
    mode='lines',
    name='Observed'
))

# Add the forecast data line, starting right after the observed data
fig.add_trace(go.Scatter(
    x=list(range(len(predicted.tolist() + forecast_mean.tolist()))),
    y=predicted.tolist() + forecast_mean.tolist(),
    mode='lines',
    line=dict(color='red'),
    name='Forecast'
))

fig.update_layout(height=500, width=1000, title = "Battery Electric Vehicle (BEV) On Road 5 Years Projection")

fig.show()


## Model - Holt's Winter

In [38]:
model_df = df[['Model Year', 'Electric Vehicle Type']].assign(count = 1).groupby(['Model Year', 'Electric Vehicle Type']).\
    count().reset_index().rename(columns = {'Model Year': 'ds', 'count': 'y'})
model_df = model_df[model_df['ds'] < 2024]
model_df['ds'] = pd.to_datetime(model_df['ds'], format='%Y')
model_df.tail()

,ds,Electric Vehicle Type,y
29,2021-01-01,Plug-in Hybrid Electric Vehicle (PHEV),3922
30,2022-01-01,Battery Electric Vehicle (BEV),23730
31,2022-01-01,Plug-in Hybrid Electric Vehicle (PHEV),4426
32,2023-01-01,Battery Electric Vehicle (BEV),52165
33,2023-01-01,Plug-in Hybrid Electric Vehicle (PHEV),7500


### Plug-in Hybrid Electric Vehicle (PHEV)

#### Filter data

In [39]:
model_df_phev = model_df[model_df['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)'][['ds', 'y']].\
    sort_values(by = 'ds')
model_df_phev.tail()

,ds,y
25,2019-01-01,2031
27,2020-01-01,1834
29,2021-01-01,3922
31,2022-01-01,4426
33,2023-01-01,7500


#### Fitting Model

In [40]:
# Fit Simple Exponential Smoothing model
series = pd.Series(model_df_phev['y'].tolist(), index=model_df_phev['ds'].tolist())
model = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=4).fit()
fitted_values = model.fittedvalues.tolist()
forecast = model.forecast(5).tolist()  

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency YS-JAN will be used.



#### Forecast visualization

In [41]:
# Create a figure
fig = go.Figure()

# Add the observed data line
fig.add_trace(go.Scatter(
    x=list(range(len(model_df_phev['y'].tolist()))),
    y=model_df_phev['y'].tolist(),
    mode='lines',
    name='Observed'
))

# Add the forecast data line, starting right after the observed data
fig.add_trace(go.Scatter(
    x=list(range(len(fitted_values + forecast))),
    y=fitted_values + forecast,
    mode='lines',
    line=dict(color='red'),
    name='Forecast'
))

fig.update_layout(height=500, width=1000, title = "Plug-in Hybrid Electric Vehicle (PHEV) On Road 5 Years Projection")

fig.show()


### Battery Electric Vehicle (BEV)

#### Filter Data

In [42]:
model_df_bev = model_df[model_df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)'][['ds', 'y']].\
    sort_values(by = 'ds')
model_df_bev.tail()

,ds,y
24,2019-01-01,8875
26,2020-01-01,10255
28,2021-01-01,15394
30,2022-01-01,23730
32,2023-01-01,52165


#### Fitting Model

In [45]:
# Fit Simple Exponential Smoothing model
series = pd.Series(model_df_bev['y'].tolist(), index=model_df_bev['ds'].tolist())
model = ExponentialSmoothing(series, trend='add', seasonal=None).fit()
fitted_values = model.fittedvalues.tolist()
forecast = model.forecast(5).tolist()  

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\soumy\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



#### Forecast visualization

In [47]:
# Create a figure
fig = go.Figure()

# Add the observed data line
fig.add_trace(go.Scatter(
    x=list(range(len(model_df_bev['y'].tolist()))),
    y=model_df_bev['y'].tolist(),
    mode='lines',
    name='Observed'
))

# Add the forecast data line, starting right after the observed data
fig.add_trace(go.Scatter(
    x=list(range(len(fitted_values + forecast))),
    y=fitted_values + forecast,
    mode='lines',
    line=dict(color='red'),
    name='Forecast'
))

fig.update_layout(height=500, width=1000, title = "Battery Electric Vehicle (BEV) On Road 5 Years Projection")

fig.show()